# Projeto final

Projeto desenvolvido para a disciplina SME0110 - Programação Matemática (2024)

Aluno: Jorge Augusto Salgado Salhani

NoUSP: 8927418

## Bibliotecas

In [37]:
!pip install mip

import pandas as pd
import random
import math
import time
from mip import *

# Leitura dos Dados

In [39]:
# Função para ler e processar os dados dos monitores e disciplinas a partir de um arquivo Excel.
# Ela lê o arquivo, limpa os nomes das colunas, remove duplicatas e extrai as colunas relevantes.
# Retorna um dicionário de alunos, uma lista de notas (Na) e uma matriz de disponibilidade (s_ad).
# Parâmetro:
# - path (str): Caminho para o arquivo Excel.
# Retornos:
# - alunos (dict): Dicionário onde a chave é o número USP e contem as seguintes informacoes --
#  -- > [Interesse em ser voluntario(ignorar), pontuacao do aluno (float), criterio de desempate (1, 2 ou 3), lista de materias que ele esta inscrito (Ex: [38, 39, 40]]).
# - Na (list): Lista das notas dos alunos.
# - s_ad (list of lists): Matriz binária de disponibilidade dos alunos para disciplinas.
def ler_dados(path):
    # Padroniza o nome das colunas, para facilitar extracao
    def limpar_nome_coluna(nome):
        return ' '.join(nome.strip().split()).lower()

    # Le e extrai as colunas necessarias
    df = pd.read_excel(path)
    df.columns = [limpar_nome_coluna(col) for col in df.columns]
    df = df.drop_duplicates(subset=['nºusp'], keep='last') # mantem apenas a ultima ocorrencia de um NUSP

    colunas_n = [
        'nºusp',
        'possui pedido de bolsa de estudos em andamento? (a concessão de bolsa de estudos implicará no cancelamento da monitoria a partir do início da vigência da bolsa)',
        'pretende se inscrever no peeg? (o acúmulo das duas monitorias não é permitido. caso o aluno seja selecionado nas duas modalidades precisará optar por uma delas)',
        'departamento de matemática (início da monitoria em 05/08/2024)',
        'departamento de matemática aplicada e estatística (início da monitoria em 05/08/2024)',
        'departamento de ciências de computação (início da monitoria em 05/08/2024)',
        'departamento de sistemas de computação (início da monitoria em 01/09/2024)',
        'tem interesse na monitoria voluntária (sem recebimento de bolsa)?',
        'média ponderada com reprovações'
    ]

    df_colunas = df[colunas_n].copy()

    # Calcula a pontuacao para criterio de desempate dos alunos (3 - mais prioridade, 2 - prioridade media, 1 - prioridade baixa)
    def calcular_pontuacao(row):
        bolsa = row[colunas_n[1]]
        peeg = row[colunas_n[2]]
        if bolsa == 'Sim' and peeg == 'Sim':
            return 1
        elif bolsa == 'Não' and peeg == 'Não':
            return 3
        return 2

    df_colunas['bolsa_peeg_status'] = df_colunas.apply(calcular_pontuacao, axis=1)
    df_colunas = df_colunas.drop([colunas_n[1], colunas_n[2]], axis=1)

    # Faz o mapeamento dos departamentos presentes na planilha
    departamento_mapping = {
        colunas_n[3]: 1,
        colunas_n[4]: 2,
        colunas_n[5]: 3,
        colunas_n[6]: 4
    }

    # Extrai as materias presentes na planilha
    materias_set = set()
    materias_dict = {}
    materia_index = 0

    for dept_col, dept_num in departamento_mapping.items():
        for materias in df_colunas[dept_col].dropna():
            for materia in materias.split(','):
                materia = materia.strip()
                if materia not in materias_set:
                    materias_set.add(materia)
                    materias_dict[materia_index] = (materia, dept_num)
                    materia_index += 1

    # Extrai os alunos, sua nota, sua pontuacao de desempate, se tem interesse em ser voluntario e as materias que ele se inscreveu
    alunos = df_colunas.set_index('nºusp').T.to_dict('list')

    Na = []
    for usp, valores in alunos.items():
        materias_combinadas = []
        valores_limpos = []
        for valor in valores:
            if isinstance(valor, str) and '-' in valor:
                for materia in valor.split(','):
                    materia = materia.strip()
                    for indice, (mat, dept) in materias_dict.items():
                        if materia == mat:
                            materias_combinadas.append(indice)
                            break
            elif isinstance(valor, str) and (valor.upper() == 'POS' or valor.upper() == 'PÓS'): # atribui a nota 1 caso o aluno for da pos, para diminuir sua prioridade
                valores_limpos.append(1)
            elif not pd.isna(valor):
                valores_limpos.append(valor)

        if materias_combinadas:
            valores_limpos.append(sorted(materias_combinadas))

        alunos[usp] = valores_limpos
        nota = valores_limpos[1] if len(valores_limpos) > 1 else None
        Na.append(nota)

    s_ad = [
        [1 if materia in valores_limpos[-1] else 0 for materia in materias_dict.keys()]
        for usp, valores_limpos in alunos.items()
    ]

    return alunos, Na, s_ad, materias_dict

# Modelo

In [40]:
def criar_modelo_fixar_atribuicao(alunos, Na, s_ad, materias, atribuicoes_fixas):
    # Conjunto de monitores (A) e disciplinas (D)
    A = alunos
    D = materias
    s = s_ad
    n_a = Na
    
    model = Model("Alocacao_de_Monitores", sense=MAXIMIZE, solver_name="CBC") 

    x = {(a, d): model.add_var(var_type=BINARY, name=f"x_{a}_{d}") for a in A for d in D}
    y = {d: model.add_var(var_type=BINARY, name=f"y_{d}") for d in D}

    # Função objetivo
    model.objective = xsum(n_a[a] * x[a, d] for a in A for d in D if a in n_a) - xsum(y[d] for d in D)

    # Sujeito a
    for d in D:
        model += xsum(x[a, d] for a in A) + y[d] == 1

    for a in A:
        model += xsum(x[a, d] for d in D) <= 1

    for a in A:
        for d in D:
            if (a, d) in s:
                model += x[a, d] <= s[a, d]

    # Nova restrição: fixando monitorias selecionadas - x_ad = 1 
    for a, d in atribuicoes_fixas:
        # Add variables if not already part of the original model
        if (a, d) not in x:
            x[(a, d)] = model.add_var(var_type=BINARY, name=f"x_{a}_{d}")
        model += x[a, d] == 1


    return {
        'model': model, 
        'x_ad': x, 
        'y_d': y, 
        'D': D
    }

In [41]:
# Função para criar o modelo de otimização de alocação de monitores com base nos dados fornecidos.
# Parâmetros:
# - alunos (dict): Dicionário de alunos (informações como notas, disponibilidade etc).
# - Na (list): Lista de notas dos alunos.
# - s_ad (list of lists): Matriz de disponibilidade dos alunos para disciplinas.
# - materias (list): Lista de todas as disciplinas.
# Retorno: Deve retornar o modelo de otimização, variáveis e os dados para análise posterior.
def criar_modelo(alunos, Na, s_ad, materias):
    # Voce deve implementar esta parte (foi deixado um exemplo de como pegar os parametros)
    # Esta é apenas uma sugestão de implementacao, voce pode mudar esta funcao se desejar ou ate criar o modelo
    # diretamente no codigo sem usar uma funcao (recomendo fortemente que nao faca isso)

    # Conjunto de monitores (A) e disciplinas (D)
    A = alunos
    D = materias
    s = s_ad
    n_a = Na

    model = Model("Alocacao_de_Monitores", sense=MAXIMIZE, solver_name="CBC") 

    x = {(a, d): model.add_var(var_type=BINARY, name=f"x_{a}_{d}") for a in A for d in D}
    y = {d: model.add_var(var_type=BINARY, name=f"y_{d}") for d in D}

    # Função objetivo
    model.objective = xsum(n_a[a] * x[a, d] for a in A for d in D if a in n_a) - xsum(y[d] for d in D)

    # Sujeito a
    for d in D:
        model += xsum(x[a, d] for a in A) + y[d] == 1

    for a in A:
        model += xsum(x[a, d] for d in D) <= 1

    for a in A:
        for d in D:
            if (a, d) in s:
                model += x[a, d] <= s[a, d]

    # return m, x_ad, y_d, D
    return {
        'model': model, 
        'x_ad': x, 
        'y_d': y, 
        'D': D
    }

# Função para resolver o modelo de otimização com base em parâmetros fornecidos.
# Parâmetros:
# - modelo (pulp.LpProblem): O modelo de otimização que será resolvido.
# - presolve (int): Se 1, presolve é ativado; se 0, é desativado.
# - cortes (int): Intensidade da geracao de cortes (0 desativa).
# Retorno: O modelo resolvido.
def resolver_modelo(modelo, presolve, cortes):
    # Configurar as opções do solver
    modelo.preprocess = presolve  # Define preprocess (-1, 0, 1)
    modelo.cuts = cortes  # Define geração de cortes (-1, 0, 1, 2, 3)

    # Medir o tempo de execução
    start_time = time.time()

    # Resolver o modelo
    status = modelo.optimize(max_seconds=1800) # Limite de tempo de 30 minutos

    # Calcular o tempo de execução
    execution_time = time.time() - start_time

    # Exibir as informações do modelo resolvido
    print(f"Solução Encontrada: {modelo.objective_value}")
    print(f"Gap: {modelo.gap * 100:.2f}%")
    print(f"Tempo de execução: {execution_time:.2f} segundos")

    return modelo

# Função para exibir as informações dos monitores alocados e não alocados, além das disciplinas não atendidas.
# Parâmetros:
# - m (pulp.LpProblem): O modelo de otimização resolvido.
# - x_ad (dict): Variáveis de decisão indicando a alocação dos monitores às disciplinas.
# - y_d (dict): Variáveis de decisão indicando disciplinas não atendidas.
# - alunos (dict): Dicionário contendo as informações dos alunos.
# - D (list): Lista de disciplinas disponíveis.
# - s_ad (list of lists): Matriz de disponibilidade dos monitores para as disciplinas.
# Retorno: Nenhum. A função imprime os resultados diretamente.
def informacoes_monitores(m, x_ad, y_d, alunos, D, s_ad):
    monitores_alocados = []
    monitores_nao_alocados = []

    total_atendidas = 0

    # Exibe os monitores alocados para disciplinas
    for (a, d) in x_ad:
        alocado = False
        if x_ad[a,d].x >= 0.99: # Verifica se a variável é 1
            monitores_alocados.append(f"Monitor {a} foi alocado para a disciplina {d}.")
            alocado = True
            total_atendidas += 1
        # Caso o monitor não tenha sido alocado, armazenar a informação para exibir depois
        if not alocado:
            disciplinas_inscrito = [d for d in D if s_ad[a,d] == 1]
            monitores_nao_alocados.append(f"Monitor {a} NÃO foi alocado. Inscrito para as disciplinas: {disciplinas_inscrito}")

    # Exibir os monitores alocados
    print("\nMonitores alocados:")
    for alocacao in monitores_alocados:
        print(alocacao)

    # Exibir os monitores não alocados
    print("\nMonitores não alocados:")
    for nao_alocacao in monitores_nao_alocados:
        print(nao_alocacao)

    # Mostrar disciplinas não atendidas e os monitores que se inscreveram para elas
    print("\nDisciplinas não atendidas e seus inscritos:")
    total_nao_atendidas_com_inscricao = 0
    for d in y_d:
        if y_d[d].x >= 0.99:  # Verifica se a variável é 1
            monitores_inscritos = [a for a in list(alunos) if s_ad[a,d] == 1]
            if monitores_inscritos:
                total_nao_atendidas_com_inscricao += 1
                print(f"A disciplina {d} não foi atendida por nenhum monitor. Inscritos: {monitores_inscritos}")
            else:
                print(f"A disciplina {d} não foi atendida por nenhum monitor e não possui inscritos.")

    print()
    print(f"Total de disciplinas atendidas: {total_atendidas}")
    print(f"Total de disciplinas não atendidas que apresentam inscritos: {total_nao_atendidas_com_inscricao}")


# Gerar Instâncias Maiores

In [42]:
# Função para aumentar os dados de alunos e matérias com base na escala fornecida.
# Parâmetros:
# - file_path (str): Caminho para o arquivo Excel com os dados originais.
# - escala (float): Fator de aumento dos dados (e.g., 1.5 para aumentar em 50%).
# Retorno: Retorna os dados aumentados (alunos, notas, disponibilidade dos alunos para disciplinas e a lista de matérias).
def aumentar_dados(path, escala):
    # Lê os dados originais usando a função existente
    alunos_originais, Na_originais, s_ad_originais, dict_materias = ler_dados(path)
    seed = 0

    # Se seed for 0 e escala for 1, retornar os dados originais
    if seed == 0 and escala == 1:
        todas_materias = list(range(len(s_ad_originais[0])))
        return {
            'alunos': alunos_originais, 
            'n_a': Na_originais, 
            's_ad': s_ad_originais, 
            'D': todas_materias, 
            'dict_materias': dict_materias
        }

    # Define a seed para a geração de dados aleatórios
    if seed is not None:
        random.seed(seed)

    novos_alunos = {}
    novas_notas = []
    nova_s_ad = []
    usp_existentes = set(alunos_originais.keys())
    max_usp = max(usp_existentes)

    # Aumenta o número de materias em 1.12x com base na escala
    num_materias_originais = len(s_ad_originais[0])
    num_novas_materias = math.ceil(num_materias_originais * 1.12 * escala)

    # Novos índices para as matérias que serão adicionadas
    novas_materias = list(range(num_materias_originais, num_materias_originais + num_novas_materias))

    # Aumenta o numero de alunos conforme a escala
    total_alunos = len(alunos_originais)
    num_novos_alunos = math.ceil(total_alunos * (escala - 1))  # Cria novos alunos além dos originais

    # Cria os novos alunos
    for i in range(num_novos_alunos):
        # Novo Nusp
        max_usp += 1
        novo_usp = max_usp
        usp_existentes.add(novo_usp)

        # Gerar uma nova nota aleatória para o novo aluno
        nova_nota = random.uniform(0, 10)
        novas_notas.append(nova_nota)

        # Gerar matérias aleatórias (entre 2 e 4) para cada novo aluno
        num_materias_aluno = random.randint(2, 4)
        todas_materias = list(range(num_materias_originais)) + novas_materias  # Considera matérias antigas e novas
        materias_aluno = random.sample(todas_materias, num_materias_aluno)

        # Criar a matriz de disponibilidade (s_ad) para o novo aluno com as matérias selecionadas
        disponibilidade_aluno = [1 if materia in materias_aluno else 0 for materia in todas_materias]
        nova_s_ad.append(disponibilidade_aluno)

        novos_alunos[novo_usp] = ['Não', nova_nota, 1, materias_aluno]

    # Expande a matriz s_ad original para acomodar as novas matérias
    s_ad_expandidos = [row + [0] * len(novas_materias) for row in s_ad_originais]
    s_ad_expandidos += nova_s_ad

    # Combinar os dados originais e os novos
    alunos_expandidos = {**alunos_originais, **novos_alunos}
    Na_expandidos = Na_originais + novas_notas

    return {
            'alunos': alunos_expandidos, 
            'n_a': Na_expandidos, 
            's_ad': s_ad_expandidos, 
            'D': todas_materias, 
            'dict_materias': dict_materias
    }

## Preâmbulo

Inicialmente devemos entender a formulação de problemas de programação matemática em solvers computacionais (neste caso, [MIP](https://docs.python-mip.com/en/latest/intro.html)). Podemos utilizar do [problema da mochila](https://docs.python-mip.com/en/latest/examples.html) dada sua simplicidade de modelagem e disponibilidade como exemplo de implementação 

Neste problema, temos um conjunto $I$ de itens, cada um apresentando um peso $w_i$ e valor agregado $p_i$. Queremos, dada uma mochila com capacidade $c$, encontrar o melhor conjunto de itens tais que não ultrapassem a capacidade da mochila e que nos retorne o maior valor agregado acumulado.

Assim, o modelo que temos é maximizar $z$ (objetivo) tal que

$$
z = \sum_{i \in I} p_i x_i
$$

sujeito a

$$
\sum_{i \in I} w_i x_i \leq c \\
x_i \in \{0,1\} \quad \forall i \in I
$$

In [43]:
from mip import Model, xsum, maximize, BINARY

p = [10, 13, 18, 31, 7, 15]     # valor agregado por item
w = [11, 15, 20, 35, 10, 33]    # peso de cada item
c, I = 47, range(len(w))        # capacidade da mochila
                                # I: intervalo contendo a quantidade de itens

m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in I]
# x: lista com I variateis do tipo binarias 
#    (decisão se item i será ou não adicionado na mochila)

m.objective = maximize(xsum(p[i] * x[i] for i in I))
# m.objective: função objetivo a ser maximizada neste caso

m += xsum(w[i] * x[i] for i in I) <= c
# Inserção no modelo m da restrição de limitação pela capacidade da mochila

m.optimize()
# Execução do processamento de otimização

selected = [i for i in I if x[i].x >= 0.99]
# Resgate das variáveis de decisão que indicam escolha do item i

print("selected items: {}".format(selected))

selected items: [0, 3]


Como será necessária contabilização de variáveis de índices duplos para a modelagem do problema deste trabalho, vale também utilizarmos do [problema de corte de Kantorovich](https://docs.python-mip.com/en/latest/examples.html). Neste problema, dado uma quantidade fixa de materia prima a ser cortada em pedaços de tamanhos conhecidos, queremos minimizar a quantidade de material de sobra.

Vamos supor que temos uma barra de tamanho $L$ e precisamos suprir a uma demanda de $n$ barras cortadas. Cada barra cortada $i$ apresenta um tamanho $w_i$ e uma demanda de $b_i$ a ser cumprida

Queremos minimizar a função objetivo $z$ dada por

$$
z = \sum_{j=1}^n y_j
$$

sujeito a

$$
\begin{align}
&\sum_{j=1}^n x_{i,j} \geq b_i \quad \forall i \in \{1,2,...,m\} \\
&\sum_{i=1}^m w_i x_{i,j} \leq L y_j \quad \forall j \in \{1,2,...,n\} \\
&y_j \in \{0,1\} \quad \forall j \in \{1,2,...,n\} \\
&x_{i,j} \in Z^+ \quad \forall i \in \{1,2,...,m\} \quad \forall j \in \{1,2,...,n\}
\end{align}
$$

Sendo $i$ o indice das demandas e $j$ o indice das barras demandadas, as restrições do modelo acima são compreendidas como

1. O total de barras cortadas $i$ deve ser ao menos a quantidade demandada $b_i$
2. O total de barras cortadas de tamanho $w_i$ deve ser menor que $L$ caso barra $j$ escolhida

In [44]:
from mip import INTEGER

n = 10  # maximum number of bars
L = 250  # bar length
m = 4  # number of requests
w = [187, 119, 74, 90]  # size of each item
b = [1, 2, 2, 1]  # demand for each item

# creating the model
model = Model()
x = {(i, j): model.add_var(obj=0, var_type=INTEGER, name="x[%d,%d]" % (i, j))
     for i in range(m) for j in range(n)}
y = {j: model.add_var(obj=1, var_type=BINARY, name="y[%d]" % j)
     for j in range(n)}

# constraints
for i in range(m):
    model.add_constr(xsum(x[i, j] for j in range(n)) >= b[i])
for j in range(n):
    model.add_constr(xsum(w[i] * x[i, j] for i in range(m)) <= L * y[j])

# additional constraints to reduce symmetry
for j in range(1, n):
    model.add_constr(y[j - 1] >= y[j])

# optimizing the model
model.optimize()

# printing the solution
print('')
print('Objective value: {model.objective_value:.3}'.format(**locals()))
print('Solution: ', end='')
for v in model.vars:
    if v.x > 1e-5:
        print('{v.name} = {v.x}'.format(**locals()))
        print('          ', end='')


Objective value: 3.0
Solution: x[0,0] = 1.0
          x[1,2] = 2.0
          x[2,1] = 2.0
          x[3,1] = 1.0
          y[0] = 1.0
          y[1] = 1.0
          y[2] = 1.0
          

Passamos agora ao modelo especificado neste trabalho

### Entendendo o modelo e o conjunto de dados

Parametros

$$
\begin{align*}
A: &\text{Conjunto de monitores, com a = \{1,2,3,...,|A|\}} \\
D: &\text{Conjunto de disciplinas, com d = \{1,2,3,...,|D|\}} \\
s_{ad}: &\text{1, se monitor 'a' disposto a monitorar disciplina 'd';} \\
&\text{0, caso contrário} \\
N_a: &\text{Nota média do monitor 'a'} \\
\end{align*}
$$

Variáveis

$$
\begin{align*}
x_{ad}: &\text{1, se monitor 'a' alocado para disciplina 'd';} \\
&\text{0 caso contrário} \\
y_d: &\text{1, se a disciplina 'd' não possui monitor;} \\
&\text{0 caso contrário}
\end{align*}
$$

Modelo

max(z) tal que
$$
z = \sum_{d \in D} \sum_{a \in A} N_a x_{ad} - \sum_{d \in D} y_d
$$

Sujeito a

$$
\begin{align}
\sum_{a \in A} x_{ad} + y_d = 1&; \quad \forall d \in D \\
\sum_{d \in D} x_{ad} \leq 1&; \quad \forall a \in A \\
x_{ad} \leq s_{ad}&; \quad \forall a \in A \quad \forall d \in D \\
x_{ad} \in \{0,1\}&; \quad \forall a \in A \quad \forall d \in D \\
y_{d} \in \{0,1\}&; \quad \forall d \in D
\end{align}
$$

A função objetivo busca maximizar a soma ponderada das médias dos monitores alocados às disciplinas, ao mesmo tempo em que minimiza o número de disciplinas não atendidas. 

A restrição (1) assegura que cada disciplina tenha, no máximo, um monitor alocado. 

A restrição (2) garante que cada monitor seja alocado a, no máximo, uma disciplina. 

A restrição (3) impõe que um monitor só seja alocado a uma disciplina se ele tiver manifestado interesse em monitorá-la. 

As restrições (4)-(5) são de domínio das variáveis.

In [45]:
import pandas as pd
df = pd.read_excel('Dados_monitores.xlsx')
df.head()

,NºUSP,Possui pedido de Bolsa de Estudos em andamento? (A concessão de Bolsa de Estudos implicará no cancelamento da monitoria a partir do início da vigência da bolsa),Pretende se inscrever no PEEG? (O acúmulo das duas monitorias não é permitido. Caso o aluno seja selecionado nas duas modalidades precisará optar por uma delas),Departamento de Matemática \n(Início da monitoria em 05/08/2024),Departamento de Matemática Aplicada e Estatística \n(Início da monitoria em 05/08/2024),Departamento de Ciências de Computação\n(Início da monitoria em 05/08/2024),Departamento de Sistemas de Computação\n(Início da monitoria em 01/09/2024),Tem interesse na monitoria voluntária (sem recebimento de bolsa)?,Média ponderada com reprovações
0,11780000,Não,Não,NaN,NaN,"01-SCC0122 - Estrutura de Dados, 02-SCC0201 - ...",NaN,Sim,6.8
1,14590001,Não,Não,"02-SLC0602 - Geometria Analítica, 08-SMA0304 -...","06-SME0300 - Cálculo Numérico (T1), 07-SME0300...","01-SCC0122 - Estrutura de Dados, 02-SCC0201 - ...",03-SSC0304 - Introdução à Programação para Eng...,Não,8.8
2,12730002,Não,Sim,NaN,NaN,16-SCC0604 - Programação Orientada a Objetos,"06-SSC0603 - Estrutura de Dados I, 07-SSC0902 ...",Sim,6.9
3,10740003,Não,Sim,NaN,NaN,"01-SCC0122 - Estrutura de Dados, 02-SCC0201 - ...",03-SSC0304 - Introdução à Programação para Eng...,Não,0
4,14740004,Não,Não,06-SMA0180 - Matemática Discreta I,NaN,02-SCC0201 - Introdução a Ciências de Computaç...,NaN,Não,8.6


### Estruturando dados para modelo

In [46]:
import re

def obterConjuntoD(df):
  colunas_departamentos = [
    'Departamento de Ciências de Computação\n(Início da monitoria em 05/08/2024)',
    'Departamento de Matemática   \n(Início da monitoria em 05/08/2024)',
    'Departamento de Matemática Aplicada e Estatística  \n(Início da monitoria em 05/08/2024)',
    'Departamento de Sistemas de Computação\n(Início da monitoria em 01/09/2024)'
  ]

  D = []

  for depto in colunas_departamentos:

    mats = list(df[depto])
    mats_nomes = [mat.split(',') for mat in mats if type(mat) is str]

    codigo_discip_mask = r"\d{2}-[a-zA-Z]{3}\d{4}"

    for mat_a in mats_nomes:
      for mat_i in mat_a:
        if not type(mat_i) is str: continue
        cod_disc = re.match(codigo_discip_mask, mat_i)
        if not cod_disc: continue
        D.append(cod_disc.group())

  return set(D)

def obterConjuntoA(df):
  return set(list(map(lambda x : str(x), df['NºUSP'])))

def obterNa(df):
  medias = list(df['Média ponderada com reprovações'])
  nroUsp = list(map(lambda x : str(x), df['NºUSP']))
  Na = {}
  [Na.update({nroUsp[i]: medias[i]}) for i in range(len(medias)) if type(medias[i]) in [float,int] ]
  return Na

def obterSad(df, D):
  colunas_departamentos = [
    'Departamento de Ciências de Computação\n(Início da monitoria em 05/08/2024)',
    'Departamento de Matemática   \n(Início da monitoria em 05/08/2024)',
    'Departamento de Matemática Aplicada e Estatística  \n(Início da monitoria em 05/08/2024)',
    'Departamento de Sistemas de Computação\n(Início da monitoria em 01/09/2024)'
  ]

  Sad = {}
  for a_i in range(len(df)):
    for depto in colunas_departamentos:
      for d in D:
        if type(df.iloc[a_i][depto]) is not str: continue
        if d in df.iloc[a_i][depto]:
          Sad.update({(str(df.iloc[a_i]['NºUSP']), d): 1})
        else:
          Sad.update({(str(df.iloc[a_i]['NºUSP']), d): 0})

  return Sad

    


In [47]:
D = obterConjuntoD(df)
A = obterConjuntoA(df)
n_a = obterNa(df)
s_ad = obterSad(df, D)

In [48]:
print(f'Disciplinas (D):\n{D}\n\n')
print(f'Alunos (A):\n{A}\n\n')
print(f'Relação nota-aluno (Na):\n{n_a}\n\n')
print(f'Relação Disciplina-Desejo-aluno (Sad):\n{s_ad}')

Disciplinas (D):
{'01-SME0110', '04-SLC0613', '12-SCC0275', '03-SLC0608', '16-SME0240', '18-SME0220', '13-SME0820', '21-SMA0804', '01-SCC0122', '03-SME0142', '07-SSC0902', '11-SMA0353', '09-SMA0332', '08-SCC0244', '05-SSC0541', '07-SMA0300', '11-SME0800', '14-SCC0502', '01-SSC0108', '06-SME0300', '14-SME0828', '06-SMA0180', '08-SMA0304', '19-SCC0607', '10-SMA0343', '09-SME0306', '04-SSC0534', '12-SME0809', '16-SCC0604', '17-SMA0378', '18-SCC0220', '07-SCC0241', '06-SSC0603', '17-SME0341', '15-SME0320', '11-SCC0260', '05-SME0241', '02-SCC0201', '02-SLC0602', '02-SSC0124', '08-SSC0958', '03-SSC0304', '15-SMA0371'}


Alunos (A):
{'90390040', '13710085', '13740007', '14590031', '14580006', '14590001', '12550032', '12680005', '13870027', '15110073', '11210018', '13670034', '11210068', '14650065', '14750091', '14560051', '14570008', '13730028', '13830083', '89570023', '12920042', '11810057', '55630067', '12550049', '14550011', '12540029', '11790052', '12680075', '12540096', '14600054', '1456

### Questão 1

In [49]:
modelo_criado = criar_modelo(A, n_a, s_ad, D)
resultado = resolver_modelo(modelo_criado['model'], 0, 0)

Solução Encontrada: 315.5
Gap: 0.00%
Tempo de execução: 0.15 segundos


In [50]:
informacoes_monitores(modelo_criado['model'], modelo_criado['x_ad'], modelo_criado['y_d'], A, D, s_ad)


Monitores alocados:
Monitor 14590031 foi alocado para a disciplina 17-SME0341.
Monitor 15110073 foi alocado para a disciplina 05-SME0241.
Monitor 14560051 foi alocado para a disciplina 09-SMA0332.
Monitor 89570023 foi alocado para a disciplina 07-SSC0902.
Monitor 11810057 foi alocado para a disciplina 10-SMA0343.
Monitor 55630067 foi alocado para a disciplina 04-SLC0613.
Monitor 12540029 foi alocado para a disciplina 16-SME0240.
Monitor 11790052 foi alocado para a disciplina 02-SLC0602.
Monitor 12540096 foi alocado para a disciplina 05-SSC0541.
Monitor 14560074 foi alocado para a disciplina 02-SCC0201.
Monitor 12550033 foi alocado para a disciplina 14-SME0828.
Monitor 14560087 foi alocado para a disciplina 07-SMA0300.
Monitor 12540058 foi alocado para a disciplina 02-SSC0124.
Monitor 11220025 foi alocado para a disciplina 17-SMA0378.
Monitor 11800060 foi alocado para a disciplina 09-SME0306.
Monitor 14570078 foi alocado para a disciplina 18-SCC0220.
Monitor 14610047 foi alocado para a

Notamos que apenas uma disciplina não foi atendida que apresentava inscritos. Além disso, dado que o valor [GAP](https://python-mip.readthedocs.io/en/latest/classes.html) representa o relação entre o custo da melhor solução encontrada e do limitante da função objetivo e GAP = 0, a solução encontrada é ótima

### Questão 2

Considerando em haver disciplinas sem monitores inscritos mas com potenciais monitores (pessoas que concluíram tal disciplina), temos dois cenários possíveis. Caso

1. existam pessoas que não foram atribuídas a nenhuma monitoria: podemos ordenar pela média ponderada e criar uma lista de potenciais monitoras a serem notificadas e convidadas à monitoria

2. contrário, ou seja, tais pessoas já apresentam atribuições a outras disciplinas: é necessário realocar uma pessoa da sua disciplina original à disciplina vaziasem monitor, mas neste caso podemos prejudicar o resultado da função objetivo. Para reduzirmos a diferença do novo valor ao ótimo encontrado via otimização podemos escolher uma pessoa que passe nos seguintes critérios

a. apresente boa média ponderada na disciplina sem monitor
b. ao ser realocada, sua disciplina original deverá ter alguém de média ponderada similar para ocupar seu lugar de modo que a realocação preserve ao máximo a função objetivo

O problema que encontramos na segunda abordagem é que a realocação gera uma cascata de realocações. Caso a série de realocações não desestruture a atribuição original, pode ser uma boa estratégia, no entanto como depende dos dados, não é garantido que isso ocorra. Nesse sentido, uma estratégia menos matemática e mais organizacional (que inclusive não é válida ao modelo matemático proposto, visto que cada pessoa pode ser atribuída ao máximo a uma disciplina pela regra 2) que pode ser tomada é a atribuição de um monitor a mais de uma disciplina com carga de dedicação dividida entre elas. Neste caso, por exemplo, onde apenas uma disciplina não apresenta inscrits, podemos selecionar a pessoa que já cursou esta disciplina com a maior média ponderada e propor que seja monitora da disciplina originalmente atribuída e também à disciplina vaga com metade da carga horária para cada uma

### Questão 3

Podemos fixar aleatoriamente as alunas às disciplinas

```json
{
  "12540096": "05-SSC0541", 
  "12680075": "15-SME0320", 
  "14560051": "08-SMA0304", 
  "10270009": "02-SSC0124", 
  "14590031": "17-SME0341"
}
```

Ao executarmos o modelo com tais atribuições fixas temos os resultados abaixo

In [51]:
num_selec = 5
monitora_discip_atribuidas = [key for key, value in s_ad.items() if value == 1]
fixadas = [
  ('12540096', '05-SSC0541'), 
  ('12680075', '15-SME0320'), 
  ('14560051', '08-SMA0304'), 
  ('10270009', '02-SSC0124'), 
  ('14590031', '17-SME0341')
  ]
print(f"Monitoras selecionadas:\n {fixadas}")

Monitoras selecionadas:
 [('12540096', '05-SSC0541'), ('12680075', '15-SME0320'), ('14560051', '08-SMA0304'), ('10270009', '02-SSC0124'), ('14590031', '17-SME0341')]


In [52]:
modelo_criado = criar_modelo_fixar_atribuicao(A, n_a, s_ad, D, fixadas)
resultado = resolver_modelo(modelo_criado['model'], 0, 0)

Solução Encontrada: 296.0
Gap: 0.00%
Tempo de execução: 0.18 segundos


In [53]:
informacoes_monitores(modelo_criado['model'], modelo_criado['x_ad'], modelo_criado['y_d'], A, D, s_ad)


Monitores alocados:
Monitor 14590031 foi alocado para a disciplina 17-SME0341.
Monitor 14560051 foi alocado para a disciplina 08-SMA0304.
Monitor 89570023 foi alocado para a disciplina 07-SSC0902.
Monitor 11810057 foi alocado para a disciplina 10-SMA0343.
Monitor 55630067 foi alocado para a disciplina 04-SLC0613.
Monitor 12540029 foi alocado para a disciplina 16-SME0240.
Monitor 11790052 foi alocado para a disciplina 02-SLC0602.
Monitor 12680075 foi alocado para a disciplina 15-SME0320.
Monitor 12540096 foi alocado para a disciplina 05-SSC0541.
Monitor 14560074 foi alocado para a disciplina 02-SCC0201.
Monitor 12550033 foi alocado para a disciplina 14-SME0828.
Monitor 14560087 foi alocado para a disciplina 07-SMA0300.
Monitor 11220025 foi alocado para a disciplina 17-SMA0378.
Monitor 11800060 foi alocado para a disciplina 09-SME0306.
Monitor 14570078 foi alocado para a disciplina 18-SCC0220.
Monitor 14610047 foi alocado para a disciplina 11-SME0800.
Monitor 13690016 foi alocado para a

Notamos que agora a função objetivo teve ser resultado reduzido, com 2 disciplinas não atendida e que apresentaram inscritos

Podemos fazer uma variação deste código para que sejam escolhidas arbitrariamente 5 pessoas a cada execução. É possível que execuções resultem em vazio, ou seja, sem resultado que respeite às restrições do problema

In [54]:
num_selec = 5
monitora_discip_atribuidas = [key for key, value in s_ad.items() if value == 1]
fixadas = [random.choice(monitora_discip_atribuidas) for i in range(num_selec)]
print(f"Monitoras selecionadas:\n {fixadas}")

modelo_criado = criar_modelo_fixar_atribuicao(A, n_a, s_ad, D, fixadas)
resultado = resolver_modelo(modelo_criado['model'], 0, 0)

informacoes_monitores(modelo_criado['model'], modelo_criado['x_ad'], modelo_criado['y_d'], A, D, s_ad)

Monitoras selecionadas:
 [('13730028', '03-SSC0304'), ('14140093', '15-SMA0371'), ('12670045', '11-SCC0260'), ('14560051', '03-SLC0608'), ('12730002', '06-SSC0603')]
Solução Encontrada: 294.80000000000007
Gap: 0.00%
Tempo de execução: 0.24 segundos

Monitores alocados:
Monitor 14590031 foi alocado para a disciplina 17-SME0341.
Monitor 15110073 foi alocado para a disciplina 05-SME0241.
Monitor 14560051 foi alocado para a disciplina 03-SLC0608.
Monitor 13730028 foi alocado para a disciplina 03-SSC0304.
Monitor 13830083 foi alocado para a disciplina 05-SSC0541.
Monitor 89570023 foi alocado para a disciplina 07-SSC0902.
Monitor 11810057 foi alocado para a disciplina 10-SMA0343.
Monitor 55630067 foi alocado para a disciplina 04-SLC0613.
Monitor 12540029 foi alocado para a disciplina 16-SME0240.
Monitor 11790052 foi alocado para a disciplina 02-SLC0602.
Monitor 14560074 foi alocado para a disciplina 02-SCC0201.
Monitor 12550033 foi alocado para a disciplina 14-SME0828.
Monitor 14560087 foi a

### Questão 4

In [55]:
# Caminho do arquivo Excel com os dados originais
path = 'Dados_monitores.xlsx'

# Aumenta os dados de alunos e matérias
dados_aumentados_lista = [
  {
    'escala': 1,
    'dados': aumentar_dados(path, escala=1)},
  {
    'escala': 2,
    'dados': aumentar_dados(path, escala=2)},
  {
    'escala': 3,
    'dados': aumentar_dados(path, escala=3)},
  {
    'escala': 5,
    'dados': aumentar_dados(path, escala=5)},
  {
    'escala': 10,
    'dados': aumentar_dados(path, escala=10)},
  {
    'escala': 15,
    'dados': aumentar_dados(path, escala=15)},
]

In [56]:
def construir_s_ad_aumentados(alunos, dict_materias):
  s_ad = {}
  for nro_usp, info in alunos.items():
    for mat_i in dict_materias.keys():
      if mat_i in info[3]:
        s_ad.update({(nro_usp, dict_materias[mat_i][0]): 1})
      else:
        s_ad.update({(nro_usp, dict_materias[mat_i][0]): 0})
  return s_ad

def construir_parametros_aumentados(dados_aumentados):
  A = list(dados_aumentados['alunos'].keys())
  D = [mat[0] for mat in dados_aumentados['dict_materias'].values()]
  n_a = dict([(nroUsp, info[1]) for nroUsp, info in dados_aumentados['alunos'].items()])
  s_ad = construir_s_ad_aumentados(dados_aumentados['alunos'], dados_aumentados['dict_materias'])

  return {
    'A': A, 'D': D, 'n_a': n_a, 's_ad': s_ad
  }

def criar_resolver_modelo_aumentado(dados_aumentados, presolve=1, cortes=1):
  print(f'\nEscala: {dados_aumentados["escala"]}')
  params_aum_1 = construir_parametros_aumentados(dados_aumentados['dados'])
  modelo_criado = criar_modelo(params_aum_1['A'], params_aum_1['n_a'], params_aum_1['s_ad'], params_aum_1['D'])
  return resolver_modelo(modelo_criado['model'], presolve=presolve, cortes=cortes)

In [61]:
modelos_aument_criados = [criar_resolver_modelo_aumentado(x) for x in dados_aumentados_lista]


Escala: 1
Solução Encontrada: 478.3
Gap: 0.00%
Tempo de execução: 0.14 segundos

Escala: 2
Solução Encontrada: 525.1030524366893
Gap: 0.00%
Tempo de execução: 0.19 segundos

Escala: 3
Solução Encontrada: 540.0755898579814
Gap: 0.00%
Tempo de execução: 0.32 segundos

Escala: 5
Solução Encontrada: 541.0418160915895
Gap: 0.00%
Tempo de execução: 0.57 segundos

Escala: 10
Solução Encontrada: 557.933719823384
Gap: 0.00%
Tempo de execução: 1.18 segundos

Escala: 15
Solução Encontrada: 563.7984023322778
Gap: 0.00%
Tempo de execução: 1.48 segundos


Em primeiro lugar vale comentarmos brevemente o que a função de aumento de dados nos retorna. Seu propósito é criar novas instâncias de dados de acordo com a escala utilizada

Sendo $e$ a escala, o número de disciplinas novas é dado por $e \times 1.12$ e o número de alunos é dado por $(e-1) \times N$, onde N é o número total de alunos originalmente.

Desse modo, é compreensível o resultado que obtemos acima, onde a escala $e$ é proporcional ao tempo de execução, visto que a base de dados é incrementada com novas instâncias conforme $e$ aumenta. Também é compreensível que o valor da função objetivo seja em média proporcional ao valor $e$, pois como seu valor é dado pela diferença entre o número de disciplinas atendidas e não atendidas e, além disso, o aumento é dado em uma escala de aproximadamente:

- $e$ novas disciplinas 
- $eN$ novos alunos, 

assim em média o número de novos alunos subrepõe ao número de novas disciplinas, sendo indicativo, portanto, que existirá (em média) ao menos um aluno por disciplina. Não havendo disciplinas não atendidas, o valor da função objetivo tende a ser diretamente proporcional à escala $e$

### Questão 5

In [58]:
modelos_aument_criados = [criar_resolver_modelo_aumentado(x, cortes=0) for x in dados_aumentados_lista]


Escala: 1
Solução Encontrada: 478.3
Gap: 0.00%
Tempo de execução: 0.18 segundos

Escala: 2
Solução Encontrada: 525.1030524366893
Gap: 0.00%
Tempo de execução: 0.20 segundos

Escala: 3
Solução Encontrada: 540.0755898579814
Gap: 0.00%
Tempo de execução: 0.25 segundos

Escala: 5
Solução Encontrada: 541.0418160915895
Gap: 0.00%
Tempo de execução: 0.58 segundos

Escala: 10
Solução Encontrada: 557.933719823384
Gap: 0.00%
Tempo de execução: 1.07 segundos

Escala: 15
Solução Encontrada: 563.7984023322778
Gap: 0.00%
Tempo de execução: 1.84 segundos


### Questão 6

In [31]:
modelos_aument_criados = [criar_resolver_modelo_aumentado(x, presolve=0) for x in dados_aumentados_lista]


Escala: 1
Solução Encontrada: 478.3
Gap: 0.00%
Tempo de execução: 0.40 segundos

Escala: 2
Solução Encontrada: 525.1030524366893
Gap: 0.00%
Tempo de execução: 1.07 segundos

Escala: 3
Solução Encontrada: 540.0755898579814
Gap: 0.00%
Tempo de execução: 2.06 segundos

Escala: 5
Solução Encontrada: 541.0418160915895
Gap: 0.00%
Tempo de execução: 5.16 segundos

Escala: 10
Solução Encontrada: 557.933719823384
Gap: 0.00%
Tempo de execução: 20.12 segundos

Escala: 15
Solução Encontrada: 563.7984023322778
Gap: 0.00%
Tempo de execução: 42.22 segundos


In [32]:
modelos_aument_criados = [criar_resolver_modelo_aumentado(x, cortes=0, presolve=0) for x in dados_aumentados_lista]


Escala: 1
Solução Encontrada: 478.3
Gap: 0.00%
Tempo de execução: 0.41 segundos

Escala: 2
Solução Encontrada: 525.1030524366893
Gap: 0.00%
Tempo de execução: 1.05 segundos

Escala: 3
Solução Encontrada: 540.0755898579814
Gap: 0.00%
Tempo de execução: 2.20 segundos

Escala: 5
Solução Encontrada: 541.0418160915895
Gap: 0.00%
Tempo de execução: 3.96 segundos

Escala: 10
Solução Encontrada: 557.933719823384
Gap: 0.00%
Tempo de execução: 14.24 segundos

Escala: 15
Solução Encontrada: 563.7984023322778
Gap: 0.00%
Tempo de execução: 30.09 segundos


### Questão 7

Parametros

$$
\begin{align*}
A: &\text{Conjunto de monitores, com a = \{1,2,3,...,|A|\}} \\
D: &\text{Conjunto de disciplinas, com d = \{1,2,3,...,|D|\}} \\
s_{ad}: &\text{1, se monitor 'a' disposto a monitorar disciplina 'd';} \\
&\text{0, caso contrário} \\
N_a: &\text{Nota média do monitor 'a'} \\
\end{align*}
$$

Variáveis

$$
\begin{align*}
x_{ad}: &\text{1, se monitor 'a' alocado para disciplina 'd';} \\
&\text{0 caso contrário} \\
y_d: &\text{1, se a disciplina 'd' não possui monitor;} \\
&\text{0 caso contrário}
\end{align*}
$$

Modelo

max(z) tal que
$$
z = \sum_{d \in D} \sum_{a \in A} N_a x_{ad} - \sum_{d \in D} y_d
$$

Sujeito a

$$
\begin{align}
\sum_{a \in A} x_{ad} + y_d = 1&; \quad \forall d \in D \\
\sum_{d \in D} x_{ad} \leq 1&; \quad \forall a \in A \\
x_{ad} \leq s_{ad}&; \quad \forall a \in A \quad \forall d \in D \\
x_{ad} \in \{0,1\}&; \quad \forall a \in A \quad \forall d \in D \\
y_{d} \in \{0,1\}&; \quad \forall d \in D
\end{align}
$$

A função objetivo busca maximizar a soma ponderada das médias dos monitores alocados às disciplinas, ao mesmo tempo em que minimiza o número de disciplinas não atendidas. 

A restrição (1) assegura que cada disciplina tenha, no máximo, um monitor alocado. 

A restrição (2) garante que cada monitor seja alocado a, no máximo, uma disciplina. 

A restrição (3) impõe que um monitor só seja alocado a uma disciplina se ele tiver manifestado interesse em monitorá-la. 

As restrições (4)-(5) são de domínio das variáveis.

In [85]:
modelo_criado = criar_modelo(A, n_a, s_ad, D)
resultado = resolver_modelo(modelo_criado['model'], 0, 0)

Solução Encontrada: 478.3
Gap: 0.00%
Tempo de execução: 0.28 segundos


In [62]:
dados = ler_dados('Dados_monitores.xlsx')
# - alunos (dict): Dicionário onde a chave é o número USP e contem as seguintes informacoes --
#  -- > [Interesse em ser voluntario(ignorar), pontuacao do aluno (float), criterio de desempate (1, 2 ou 3), lista de materias que ele esta inscrito (Ex: [38, 39, 40]]).
# - Na (list): Lista das notas dos alunos.
# - s_ad (list of lists): Matriz binária de disponibilidade dos alunos para disciplinas.
alunos, na, sad, materias = dados

In [63]:
# EXEMPLO FORNECIDO

# Caminho do arquivo Excel com os dados originais
path = 'Dados_monitores.xlsx'

# Aumenta os dados de alunos e matérias
dados_aument = aumentar_dados(path, escala=1)
# alunos, Na, s_ad, materias = dados_aument


# Cria o modelo de otimização com os dados aumentados
# mod, x_ad, y_d, D = criar_modelo(alunos, Na, s_ad, materias)
# mod, D = criar_modelo(alunos, Na, s_ad, materias)

# # Resolve o modelo
# m = resolver_modelo(mod, 1, 1)

# informacoes_monitores(m, x_ad, y_d, alunos, D, s_ad)